## <center>Analyse des offres marchés publics au maroc

 In this notebook, we're going to collect information about public markets in Morocco. We'll start by gathering data and extracting text from minutes (PVs : procès verbal)  or similar documents. Once we have the necessary information, we'll move on to analyzing these markets by answering a few questions. These questions include identifying the competitors, determining the market price, and understanding the purpose of the offer. 

In [1]:
import urllib.request
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import json
import time
import pandas as pd
import numpy as np

In [51]:
# driver = webdriver.Chrome("/h?"ome/hennani/Downloads/old_dow//chromedriver_linux64 (1)/chromedriver")
driver = webdriver.Chrome("/home/hennani/Downloads/chromedriver_linux64/chromedriver")

/tmp/ipykernel_25898/389086471.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/home/hennani/Downloads/chromedriver_linux64/chromedriver")


In [53]:
driver.get("https://www.marchespublics.gov.ma/index.php?page=entreprise.EntrepriseAdvancedSearch&AllAnn")/''

In [4]:
from selenium.webdriver.support.ui import Select

In [43]:
LancerRecherche_xpath='//*[@id="ctl0_CONTENU_PAGE_AdvancedSearch_lancerRecherche"]'

ListePageSizeTop_xpath='//*[@id="ctl0_CONTENU_PAGE_resultSearch_listePageSizeTop"]'

driver.find_element(by=By.XPATH,value=LancerRecherche_xpath).click()

select=Select(driver.find_element(by=By.XPATH,value=ListePageSizeTop_xpath))

select.select_by_value('500')

In [3]:
# Xpath='//*[@id="tabNav"]/div[2]/div[2]/table/tbody/tr'

Xpath2='//*[@id="tabNav"]/div[2]/div[2]/table/tbody/tr/td[6]/div/a[1]'

Xpath3='//*[@id="ctl0_CONTENU_PAGE_resultSearch_PagerTop_ctl2"]/img'

In [2]:
from selenium.common.exceptions import NoSuchElementException  

def check_exists_by_xpath(xpath):
    
    try:
        driver.find_element(by=By.XPATH,value=xpath)
        
    except NoSuchElementException:
        
        return False
    
    return True

def get_element(Xpath):
    
        if check_exists_by_xpath(Xpath):
            
             elemenet=driver.find_element(by=By.XPATH,value=Xpath).text
        else:
            
             elemenet=''
                
        return elemenet


In [17]:
links=[]
for i in range(0,10):#20*500=10000 marches 
    
    elemenets=driver.find_elements(by=By.XPATH,value=Xpath2)
    
    for j in elemenets:
        
        ele=j.find_element(by=By.XPATH,value='//*[@id="tabNav"]/div[2]/div[2]/table/tbody/tr/td[2]/div[4]').text
        
        links.append(j.get_attribute('href'))
        
    driver.find_element(by=By.XPATH,value=Xpath3).click()

In [5]:
List_Marches_Xpath = '//*[@id="tabNav"]/div[2]/div[2]/table/tbody/tr'

In [56]:
from tqdm import tqdm

List_of_public_marckets = []

Dict = {}

number_of_Marches_to_be_scraoed = 500

number_of_Marches=0

progress_bar = tqdm(total=number_of_Marches_to_be_scraoed)

while(number_of_Marches<number_of_Marches_to_be_scraoed):
    
    List_Elemenets=driver.find_elements(by=By.XPATH,value = List_Marches_Xpath)

    for Element in List_Elemenets:

        link=Element.find_element(by=By.XPATH,value='td[6]/div/a[1]').get_attribute('href')

        Procédure=Element.find_element(by=By.XPATH,value='td[2]/div[1]').text

        Catégorie=Element.find_element(by=By.XPATH,value='td[2]/div[3]').text

        DateDePublication=Element.find_element(by=By.XPATH,value='td[2]/div[4]').text

        Référence=Element.find_element(by=By.XPATH,value='td[3]/div/div[1]/span[1]').text

        Objet=Element.find_element(by=By.XPATH,value='td[3]/div/div[2]').text

        AcheteurPublic=Element.find_element(by=By.XPATH,value='td[3]/div/div[3]').text

        LieuDexécution=Element.find_element(by=By.XPATH,value='td[4]/div/div[1]').text

        Dict={'Procédure':Procédure,

          'Catégorie':Catégorie,

          'DateDePublication':DateDePublication,

          'Référence':Référence,

          'Objet':Objet,

          'AcheteurPublic':AcheteurPublic,

          'LieuDexécution':LieuDexécution,

          'link':link

         }

        List_of_public_marckets.append(Dict)

        number_of_Marches+=1
        
        progress_bar.update(1)
        
        if Element == List_Elemenets[-1] and number_of_Marches < number_of_Marches_to_be_scraoed : 
            
                # Navigate to next page 
                
                driver.find_element(by=By.XPATH,value='//*[@id="ctl0_CONTENU_PAGE_resultSearch_PagerTop_ctl2"]/img').click()

progress_bar.close()



100%|██████████████████████████████████| 500/500 [06:39<00:00,  1.25it/s]


In [3]:
import pandas as pd

df = pd.DataFrame(List_of_public_marckets)

In [220]:
df

,Procédure,Catégorie,DateDePublication,Référence,Objet,AcheteurPublic,LieuDexécution,link
0,AOO ...,Fournitures,27/01/2024,50/DGSN/DEB/DF/SA/2023,Objet : l’acquisition de matériel de dortoir a...,Acheteur public : DIRECTION GENERALE DE LA SUR...,IFRANE,https://www.marchespublics.gov.ma/?page=entrep...
1,AOO ...,Fournitures,27/01/2024,51/DGSN/DEB/DF/SA/2023,Objet : l’acquisition de matériel de couchage ...,Acheteur public : DIRECTION GENERALE DE LA SUR...,RABAT,https://www.marchespublics.gov.ma/?page=entrep...
2,AOO ...,Fournitures,27/01/2024,68/DGSN/DEB/DF/SA/2023,Objet : la fourniture de produits alimentaires...,Acheteur public : DIRECTION GENERALE DE LA SUR...,TANGER-ASSILAH,https://www.marchespublics.gov.ma/?page=entrep...
3,AOO ...,Fournitures,26/01/2024,106/2023/S2A,Objet : FOURNITURE ET INSTALLATION DU MATERIEL...,Acheteur public : Société de Développement Loc...,MAROC\nAGADIR IDA OU TANANE\n...,https://www.marchespublics.gov.ma/?page=entrep...
4,AOO ...,Travaux,26/01/2024,118/2023/S2A,Objet : TRAVAUX DE REALISATION DU PARC URBAIN ...,Acheteur public : Société de Développement Loc...,MAROC\nAGADIR IDA OU TANANE\n...,https://www.marchespublics.gov.ma/?page=entrep...
...,...,...,...,...,...,...,...,...
495,AOO ...,Services,18/01/2024,3/AOO/DPEDNPSZ/DG/ANP/2023,Objet : PRESTATIONS DE BUREAU D'ETUDE TECHNIQU...,Acheteur public : AGENCE NATIONALE DES PORTS,MAROC\nCASABLANCA\n...,https://www.marchespublics.gov.ma/?page=entrep...
496,AOO ...,Services,18/01/2024,TEST1/2023/ADS,Objet : L’hébergement du site web institutionn...,Acheteur public : AGENCE DE DEVELOPPEMENT SOCIAL,RABAT,https://www.marchespublics.gov.ma/?page=entrep...
497,AOO ...,Services,18/01/2024,37/DRPC/ANP/2023,Objet : REMISE EN ETAT DES STRUCTURES METALLIQ...,Acheteur public : AGENCE NATIONALE DES PORTS/ ...,CASABLANCA,https://www.marchespublics.gov.ma/?page=entrep...
498,AOO ...,Services,18/01/2024,37/DRPC/ANP/2023,Objet : REMISE EN ETAT DES STRUCTURES METALLIQ...,Acheteur public : AGENCE NATIONALE DES PORTS/ ...,CASABLANCA,https://www.marchespublics.gov.ma/?page=entrep...


In [24]:
df.to_csv('Marches_publics.csv',index=False)

The upcoming script aims to download minute files. After that, we will extract the text from these files using OCR (Optical Character Recognition). Once the text has been collected, the script will remove the file before downloading the next file to the specified directory

In [21]:
# pip install tesseract-ocr
# pip install pytesseract
# !pip install PyPDF2
# !pip install nbconvert
# !pip install pdf2image 
# !pip install python-docx

In [2]:
import pytesseract
import os
import PyPDF2
# import docx
from pdf2image import convert_from_path
custom_config = r'--oem 3 --psm 6'

In [8]:
xpath_link_to_the_='//*[@id="ctl0_CONTENU_PAGE_panelOnglet1"]/div[2]/div[1]/div[2]/div[1]/div[2]/div[1]/ul/li[1]/a'

In [14]:
# Set the download path

download_path = '/home/hennani/Downloads/marches_pv'

# Configure Chrome options

chrome_options = Options()

chrome_options.add_experimental_option('prefs', {
    
    'download.default_directory': download_path,
    
    'download.prompt_for_download': False,
    
    'plugins.plugins_disabled': ['Chrome PDF Viewer']
    
})

# Create a new instance of the Chrome driver

driver = webdriver.Chrome("/home/hennani/Downloads/chromedriver_linux64/chromedriver",options=chrome_options)



/tmp/ipykernel_108805/2813635837.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/home/hennani/Downloads/chromedriver_linux64/chromedriver",options=chrome_options)


In [3]:

from PIL import Image 
Image.MAX_IMAGE_PIXELS = 260000000 

In [26]:
Extracted_text = []

for i in range(len(df)): #len(df) 
    
#     print("=====================================",i,"======================================")
    
    driver.get(df['link'][i])

    file_link=driver.find_element(by=By.XPATH,value=xpath_link_to_the_)

    dowload_link=file_link.get_attribute('href')

    driver.get(dowload_link)
    
    # Wait for the file to load, adjust the time as needed based on the file size and network speed
    
    
    # Check if the file exists / until the File has been downloaded
    
    time.sleep(3)
    
    while  ( os.listdir('/home/hennani/Downloads/marches_pv')[-1].lower().startswith('.com.google.Chrome')
        
           or ( os.listdir('/home/hennani/Downloads/marches_pv')[-1].lower().endswith('crdownload'))): 
        
#         print(os.listdir('/home/hennani/Downloads/marches_pv')[-1].lower().endswith('crdownload'))

        time.sleep(5)
        
#         print("Waiting for file to be downloaded...")
    
    # list of files / one file !!!!
    
    files = os.listdir('/home/hennani/Downloads/marches_pv')
    
    
#     files[0] we have only one file
    
    if  files[0].lower().endswith(('.png', '.jpg', '.jpeg')):
        
        img =r'/home/hennani/Downloads/marches_pv/'+files[0]
        
        image_string = pytesseract.image_to_string(img, config=custom_config)
        
        df.loc[i, 'EXTRAIT_DE_PV'] = image_string 
        
        Extracted_text.append(image_string)
        
    elif files[0].lower().endswith(('.pdf')):
        
        pdf_File_Object = open('/home/hennani/Downloads/marches_pv/'+files[-1], 'rb')

        pdf_Reader = PyPDF2.PdfFileReader(pdf_File_Object,strict=False)
        
        # extract text from pdf with a number of pages under of 2
        
        if (pdf_Reader.numPages)<=2:
            
                pdfs = r"/home/hennani/Downloads/marches_pv/"+files[-1]
                
                pages = convert_from_path(pdfs, 350)
                
                imagestring=''
                
                for page in pages:
                    
                    imagestring+=pytesseract.image_to_string(page, config=custom_config)
                
                Extracted_text.append(imagestring)
                
                df.loc[i, 'EXTRAIT_DE_PV'] = imagestring 

    else :
        
        Extracted_text.append('')
        
        df.loc[i, 'EXTRAIT_DE_PV'] = '' 
    
    if os.path.exists('/home/hennani/Downloads/marches_pv/'+files[-1]):
        
        # Delete the file
        
        os.remove('/home/hennani/Downloads/marches_pv/'+files[-1])



In [ ]:
# df.to_csv('Marches_publics.csv',index=False)

In [4]:
df = pd.read_csv('Marches_publics.csv')

df

,Procédure,Catégorie,DateDePublication,Référence,Objet,AcheteurPublic,LieuDexécution,link,EXTRAIT_DE_PV
0,AOO ...,Fournitures,27/01/2024,50/DGSN/DEB/DF/SA/2023,Objet : l’acquisition de matériel de dortoir a...,Acheteur public : DIRECTION GENERALE DE LA SUR...,IFRANE,https://www.marchespublics.gov.ma/?page=entrep...,ROVAUME DU MAROC\nMINISTERE DE L’INTERIEUR\nDI...
1,AOO ...,Fournitures,27/01/2024,51/DGSN/DEB/DF/SA/2023,Objet : l’acquisition de matériel de couchage ...,Acheteur public : DIRECTION GENERALE DE LA SUR...,RABAT,https://www.marchespublics.gov.ma/?page=entrep...,ROVAUME DU MAROC\nMINISTERE DE L’INTERIEUR\nDI...
2,AOO ...,Fournitures,27/01/2024,68/DGSN/DEB/DF/SA/2023,Objet : la fourniture de produits alimentaires...,Acheteur public : DIRECTION GENERALE DE LA SUR...,TANGER-ASSILAH,https://www.marchespublics.gov.ma/?page=entrep...,ROYAUME DU MAROC\nMINISTERE DE L’INTERIEUR\nDI...
3,AOO ...,Fournitures,26/01/2024,106/2023/S2A,Objet : FOURNITURE ET INSTALLATION DU MATERIEL...,Acheteur public : Société de Développement Loc...,MAROC\nAGADIR IDA OU TANANE\n...,https://www.marchespublics.gov.ma/?page=entrep...,Royaume du Maroc\n\\noQoNEO O80 CoOOc oLElof[C...
4,AOO ...,Travaux,26/01/2024,118/2023/S2A,Objet : TRAVAUX DE REALISATION DU PARC URBAIN ...,Acheteur public : Société de Développement Loc...,MAROC\nAGADIR IDA OU TANANE\n...,https://www.marchespublics.gov.ma/?page=entrep...,Royaume du Maroc\n\\noQoNEO O80 CoOOc oLElof[C...
...,...,...,...,...,...,...,...,...,...
495,AOO ...,Services,18/01/2024,3/AOO/DPEDNPSZ/DG/ANP/2023,Objet : PRESTATIONS DE BUREAU D'ETUDE TECHNIQU...,Acheteur public : AGENCE NATIONALE DES PORTS,MAROC\nCASABLANCA\n...,https://www.marchespublics.gov.ma/?page=entrep...,ROYAUME DU MAROC\nExtrait du procés-verbal de ...
496,AOO ...,Services,18/01/2024,TEST1/2023/ADS,Objet : L’hébergement du site web institutionn...,Acheteur public : AGENCE DE DEVELOPPEMENT SOCIAL,RABAT,https://www.marchespublics.gov.ma/?page=entrep...,NaN
497,AOO ...,Services,18/01/2024,37/DRPC/ANP/2023,Objet : REMISE EN ETAT DES STRUCTURES METALLIQ...,Acheteur public : AGENCE NATIONALE DES PORTS/ ...,CASABLANCA,https://www.marchespublics.gov.ma/?page=entrep...,a\n|\nINN\nLE 12/01/2024 ANP-DRPC ;\nEXTRAIT D...
498,AOO ...,Services,18/01/2024,37/DRPC/ANP/2023,Objet : REMISE EN ETAT DES STRUCTURES METALLIQ...,Acheteur public : AGENCE NATIONALE DES PORTS/ ...,CASABLANCA,https://www.marchespublics.gov.ma/?page=entrep...,oO _\n= ot S ‘ z |\nwV —- + < k\nVQ A 4 ¢ .\\n...


In [30]:
file = os.listdir('/home/hennani/Downloads/marches_pv')

file

[]

In [38]:
filtered_df = df[~df['EXTRAIT_DE_PV'].isnull()]

In [44]:
filtered_df.to_csv('Marches_publics_.csv',index=False)

In [48]:
df1 = pd.read_csv("Marches_publics_.csv")

In [49]:
df1


,Procédure,Catégorie,DateDePublication,Référence,Objet,AcheteurPublic,LieuDexécution,link,EXTRAIT_DE_PV
0,AOO ...,Fournitures,27/01/2024,50/DGSN/DEB/DF/SA/2023,Objet : l’acquisition de matériel de dortoir a...,Acheteur public : DIRECTION GENERALE DE LA SUR...,IFRANE,https://www.marchespublics.gov.ma/?page=entrep...,ROVAUME DU MAROC\nMINISTERE DE L’INTERIEUR\nDI...
1,AOO ...,Fournitures,27/01/2024,51/DGSN/DEB/DF/SA/2023,Objet : l’acquisition de matériel de couchage ...,Acheteur public : DIRECTION GENERALE DE LA SUR...,RABAT,https://www.marchespublics.gov.ma/?page=entrep...,ROVAUME DU MAROC\nMINISTERE DE L’INTERIEUR\nDI...
2,AOO ...,Fournitures,27/01/2024,68/DGSN/DEB/DF/SA/2023,Objet : la fourniture de produits alimentaires...,Acheteur public : DIRECTION GENERALE DE LA SUR...,TANGER-ASSILAH,https://www.marchespublics.gov.ma/?page=entrep...,ROYAUME DU MAROC\nMINISTERE DE L’INTERIEUR\nDI...
3,AOO ...,Fournitures,26/01/2024,106/2023/S2A,Objet : FOURNITURE ET INSTALLATION DU MATERIEL...,Acheteur public : Société de Développement Loc...,MAROC\nAGADIR IDA OU TANANE\n...,https://www.marchespublics.gov.ma/?page=entrep...,Royaume du Maroc\n\\noQoNEO O80 CoOOc oLElof[C...
4,AOO ...,Travaux,26/01/2024,118/2023/S2A,Objet : TRAVAUX DE REALISATION DU PARC URBAIN ...,Acheteur public : Société de Développement Loc...,MAROC\nAGADIR IDA OU TANANE\n...,https://www.marchespublics.gov.ma/?page=entrep...,Royaume du Maroc\n\\noQoNEO O80 CoOOc oLElof[C...
...,...,...,...,...,...,...,...,...,...
380,AOO ...,Services,18/01/2024,4/AOO/DPEDNPSZ/DG/ANP/2023,Objet : CONTROLE TOPOGRAPHIQUE DES TRAVAUX DE ...,Acheteur public : AGENCE NATIONALE DES PORTS,MAROC\nCASABLANCA\n...,https://www.marchespublics.gov.ma/?page=entrep...,ROYAUME DU MAROC\nExtrait du procés-verbal de ...
381,AOO ...,Services,18/01/2024,3/AOO/DPEDNPSZ/DG/ANP/2023,Objet : PRESTATIONS DE BUREAU D'ETUDE TECHNIQU...,Acheteur public : AGENCE NATIONALE DES PORTS,MAROC\nCASABLANCA\n...,https://www.marchespublics.gov.ma/?page=entrep...,ROYAUME DU MAROC\nExtrait du procés-verbal de ...
382,AOO ...,Services,18/01/2024,37/DRPC/ANP/2023,Objet : REMISE EN ETAT DES STRUCTURES METALLIQ...,Acheteur public : AGENCE NATIONALE DES PORTS/ ...,CASABLANCA,https://www.marchespublics.gov.ma/?page=entrep...,a\n|\nINN\nLE 12/01/2024 ANP-DRPC ;\nEXTRAIT D...
383,AOO ...,Services,18/01/2024,37/DRPC/ANP/2023,Objet : REMISE EN ETAT DES STRUCTURES METALLIQ...,Acheteur public : AGENCE NATIONALE DES PORTS/ ...,CASABLANCA,https://www.marchespublics.gov.ma/?page=entrep...,oO _\n= ot S ‘ z |\nwV —- + < k\nVQ A 4 ¢ .\\n...
